# Create and run Earthquake Estimator

In [1]:
!nvidia-smi

Mon Feb  4 14:31:02 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:81:00.0 Off |                    0 |
| N/A   51C    P0    33W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import tensorflow as tf
import lstm_estimator
import earthquake_input_fn
import lstm_estimator_4096
from importlib import reload
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os.path as osp
import pickle
%matplotlib inline

In [3]:
def reload_tf():
    global lstm_estimator
    global earthquake_input_fn
    global lstm_estimator_4096
    for i in range(2):
        import lstm_estimator
        import earthquake_input_fn
        import lstm_estimator_4096
        reload(lstm_estimator)
        reload(earthquake_input_fn)
        reload(lstm_estimator_4096)
        del lstm_estimator
        del earthquake_input_fn
        del lstm_estimator_4096
    import lstm_estimator
    import earthquake_input_fn
    import lstm_estimator_4096
reload_tf()

In [4]:
def create_estimator_4096_from_params(batch_size, timesteps, n_feats, feature_columns,
                                      lstm_cell_size, dense_size, cnn_size, learning_rate, dropout_rate=0.5, grad_clip=0.5,
                                      time_pool=8, lambda_l2_reg=0.0005, stft_frame_length=512, stft_frame_step=64,
                                      label_input_column=None, model_dir=None, eval_every=600, use_stft=True, use_stride=True,
                                      regularize_networks=True, use_dense_batch_norm=True, optimizer_name='Adam',
                                      lstm_directionality='bidirectional'):
    
    params = {
        'batch_size': batch_size,
        'timesteps': timesteps,
        'n_feats': n_feats,
        'feature_columns': feature_columns,
        'lstm_cell_size': lstm_cell_size,
        'dense_size': dense_size,
        'cnn_size': cnn_size,
        'learning_rate': learning_rate,
        'dropout_rate': dropout_rate,
        'grad_clip': grad_clip,
        'lambda_l2_reg': lambda_l2_reg,
        'stft_frame_length': stft_frame_length,
        'stft_frame_step': stft_frame_step,
        'label_input_column': label_input_column,
        'time_pool': time_pool,
        'use_stft': use_stft,
        'use_stride': use_stride,
        'regularize_networks': regularize_networks,
        'use_dense_batch_norm': use_dense_batch_norm,
        'optimizer_name': optimizer_name,
        'lstm_directionality': lstm_directionality
    }
    if model_dir is None:
        model_dir = '/workspace/persistent-data/models/%s' % (time.strftime('%Y-%m-%d-%H-%M-%S'))
    
    params_file = osp.join(osp.split(model_dir)[0], osp.split(model_dir)[1] + '.params.pickle')
    if not osp.isfile(params_file):
        print('writing params to %s' % params_file)
        with open(params_file, 'wb') as f:
            pickle.dump(params, f)
        
    config = tf.estimator.RunConfig(model_dir=model_dir,
                                    log_step_count_steps=int(500 / batch_size),
                                    save_checkpoints_secs=300,
                                    session_config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.6666)))
    
    estim = tf.estimator.Estimator(model_fn=lstm_estimator_4096.lstm_4096_model_fn,
                                   params=params,
                                   model_dir=model_dir,
                                   config=config)
    return estim

In [ ]:
def train_and_evaluate_estimator_4096(earthquake_data_dir, eval_every_n_secs, eval_count, batch_size,
                                      timesteps, n_feats, feature_columns, lstm_cell_size, dense_size, cnn_size,
                                      learning_rate, dropout_rate, grad_clip, time_pool, lambda_l2_reg,
                                      stft_frame_length, stft_frame_step, use_stft, use_stride, regularize_networks,
                                      use_dense_batch_norm, optimizer_name, lstm_directionality, model_dir=None):
    
    estim = create_estimator_4096_from_params(batch_size, timesteps, n_feats, feature_columns=feature_columns,
                                              lstm_cell_size=lstm_cell_size, dense_size=dense_size, cnn_size=cnn_size,
                                              learning_rate=learning_rate, dropout_rate=dropout_rate, grad_clip=grad_clip,
                                              time_pool=time_pool, lambda_l2_reg=lambda_l2_reg, stft_frame_length=stft_frame_length,
                                              stft_frame_step=stft_frame_step, label_input_column=None, model_dir=model_dir,
                                              eval_every=eval_every_n_secs, use_stft=use_stft, use_stride=use_stride,
                                              regularize_networks=regularize_networks, use_dense_batch_norm=use_dense_batch_norm,
                                              optimizer_name=optimizer_name, lstm_directionality=lstm_directionality)
    
    trainspec = tf.estimator.TrainSpec(input_fn=lambda: earthquake_input_fn.earthquake_input_fn2(earthquake_data_dir,
                                                                             batch_size,
                                                                             timesteps,
                                                                             traintest='train'),
                                       max_steps=1000000)
    
    evalspec = tf.estimator.EvalSpec(input_fn=lambda: earthquake_input_fn.earthquake_input_fn2(earthquake_data_dir,
                                                                           batch_size,
                                                                           timesteps,
                                                                           traintest='test'),
                                     steps=eval_count,
                                     start_delay_secs=1, throttle_secs=eval_every_n_secs)
    
    return estim, trainspec, evalspec

In [ ]:
reload_tf()

EARTHQUAKE_DATA_DIR = '/workspace/persistent-data/earthquake/tfrecords3'
BATCH_SIZE=32
TIMESTEPS=75000
LSTM_CELL_SIZE=[512, 512]
CNN_SIZE=[[32, 4], [32, 4], [16, 6], [12, 6], [8, 8]]
DENSE_SIZE=[512, 512, 512, 4]
STFT_FRAME_LENGTH=1024
STFT_FRAME_STEP=300
LEARNING_RATE=0.0005
DROPOUT_RATE=0.5
GRAD_CLIP=1
TIME_POOL=300
LAMBDA_L2_REG=0.00005
EVAL_EVERY_N_SECONDS = 600
EVAL_NUM_BATCHES = 500
N_FEATS = 1
USE_STFT = True
USE_STRIDE = True
REGULARIZE_NETWORKS = False
USE_DENSE_BATCH_NORM = False
OPTIMIZER_NAME = 'SGD'
LSTM_DIRECTIONALITY = 'unidirectional'
FEATURE_COLUMNS = [tf.feature_column.numeric_column(key='acousticdata', dtype=tf.float32, shape=(TIMESTEPS,))]

estim, train_spec, eval_spec = train_and_evaluate_estimator_4096(EARTHQUAKE_DATA_DIR, EVAL_EVERY_N_SECONDS, EVAL_NUM_BATCHES,
                                                                 BATCH_SIZE, TIMESTEPS, N_FEATS, FEATURE_COLUMNS,
                                                                 LSTM_CELL_SIZE, DENSE_SIZE, CNN_SIZE, LEARNING_RATE,
                                                                 DROPOUT_RATE, GRAD_CLIP, TIME_POOL, LAMBDA_L2_REG,
                                                                 STFT_FRAME_LENGTH, STFT_FRAME_STEP, USE_STFT, USE_STRIDE,
                                                                 REGULARIZE_NETWORKS, USE_DENSE_BATCH_NORM, OPTIMIZER_NAME,
                                                                 LSTM_DIRECTIONALITY,
                                                                 #model_dir='/workspace/persistent-data/models/2019-02-04-04-38-07'
                                                                 )

writing params to /workspace/persistent-data/models/2019-02-04-14-31-11.params.pickle
INFO:tensorflow:Using config: {'_is_chief': True, '_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2056738668>, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.6666
}
, '_keep_checkpoint_every_n_hours': 10000, '_experimental_distribute': None, '_log_step_count_steps': 15, '_global_id_in_cluster': 0, '_master': '', '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_num_worker_replicas': 1, '_device_fn': None, '_model_dir': '/workspace/persistent-data/models/2019-02-04-14-31-11', '_train_distribute': None, '_evaluation_master': '', '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 300, '_protocol': None, '_task_id': 0, '_task_type': 'worker', '_eval_distribute': None}


In [ ]:
for i in range(100):
    try:
        print('---------------- BEGIN EPOCH %d ---------------' % i)
        tf.estimator.train_and_evaluate(estim, train_spec, eval_spec)
    except KeyboardInterrupt:
        break
    #except:
    #    print('end of epoch')

---------------- BEGIN EPOCH 0 ---------------
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 300.
INFO:tensorflow:Calling model_fn.
FEATURES
{'acousticdata': <tf.Tensor 'IteratorGetNext:0' shape=(?, 75000) dtype=float32>}
--------------------
LABELS
Tensor("IteratorGetNext:1", shape=(?, 75000), dtype=float64, device=/device:CPU:0)
--------------------
MODE
train
--------------------
PARAMS
{'lstm_cell_size': [512, 512], 'cnn_size': [[32, 4], [32, 4], [16, 6], [12, 6], [8, 8]], 'grad_clip': 1, 'use_stride': True, 'label_input_column': None, 'timesteps': 75000, 'use_dense_batch_norm': False, 'n_feats': 1, 'lstm_directionality': 'unidirectional', 'dropout_rate': 0.5, 'stft_frame_length': 1024, 'lambda